In [1]:
import subprocess as sp
import gurobipy as gp
import sage.all as sg
from sage.all import libgap as lg
import re

In [2]:
def write_orbits(orbits):
    with open("./sage_orbits.txt", "w") as f:
        idx = 0
        for orbit in orbits:
            for el in orbit:
                f.write(str(el - 1) + " ")
            f.write("\n")
            idx += 1
    print("Orbits Written")
            
            
def get_stabilizer_group_orbits(group):
    group_orbits = group.orbits()
    for orb in group_orbits:
        if len(orb) > 1:
            stable_pnt = orb[0]
            stabilizer_group = group.stabilizer(stable_pnt)
            print("New Orbits Calculated")
            return stabilizer_group, stabilizer_group.orbits()
    print("Discrete Orbits Calculated")
    return group, group.orbits()
        
def concatenate_orbits(real_orbits, cut_orbits):
    out = []
    for orb in real_orbits:
        out.append(orb)
    for orb in cut_orbits:
        out.append(orb)
    print("Orbits Concatenated")
    return out

def call_coin_cgl(lp_path):
    cgl_cut_file = "./CutGeneration/cut_txt_files/cuts.txt"
    cgl_out = sp.Popen(["./CutGeneration/generateMIPCuts",lp_path,"./sage_orbits.txt"], shell = False, 
                       stdout = sp.DEVNULL)
    cgl_return, cgl_err = cgl_out.communicate()
    print("Cgl Solved Aggregate LP")
    return cgl_return, cgl_err

def generate_cut_orbits(gap_perm_group, lp_path, cut_orbits):
    cut_rhs = []
    cut_added_model = gp.read(lp_path)
    cols = cut_added_model.getVars()
    rows = cut_added_model.getConstrs()
    num_nodes = len(cols) + len(rows) + 1
    with open("./CutGeneration/cut_txt_files/cuts.txt") as cuts_f:
        lines = cuts_f.readlines()
        for line in lines:
            splt = line.split(",")
            cut = []
            rhs = (sg.RealNumber(splt[-1][:-1]))
            sense = splt[-2]
            for coeff in splt[:-2]:
                cut.append(sg.RealNumber(coeff))
            symmetric_cuts = lg.Orbit(gap_perm_group, cut, lg.Permuted)
            new_orbits = []
            for sym_cut in symmetric_cuts:
                if sense == "L":
                    expr = sum(sym_cut[i].sage() * orig_cols[i] for i in range(len(sym_cut))) <= rhs
                elif sense == "G":
                    expr = sum(sym_cut[i].sage() * orig_cols[i] for i in range(len(sym_cut))) >= rhs
                else:
                    expr = sum(sym_cut[i].sage() * orig_cols[i] for i in range(len(sym_cut))) == rhs
                orig_model.addConstr(expr)
                new_orbits.append(num_nodes)
                num_nodes += 1
            cut_orbits.append(new_orbits)
    orig_model.write("test.mps")
    orig_model.write("test.lp")
    print("Cut Orbits Generated")

In [3]:
# Read in mps file and create dicitonary for variable names and indices 
symm_lp = "../EQLPSolver/HS-COV-COD/cod103.mps";
orig_model = gp.read(symm_lp)
orig_cols = orig_model.getVars()
orig_rows = orig_model.getConstrs()
col_idx = {col.varName : i for i, col in enumerate(orig_cols)}
col_name = {i : col.varName for i, col in enumerate(orig_cols)}
num_col = orig_model.numVars
row_idx = {row.constrName : i + num_col for i , row in enumerate(orig_rows)}
row_name = {i + num_col : row.constrName for i, row in enumerate(orig_rows)}
num_tot = len(orig_cols) + len(orig_rows)


Restricted license - for non-production use only - expires 2023-10-25
Read MPS format model from file ../EQLPSolver/HS-COV-COD/cod103.mps
Reading time = 0.00 seconds
obj: 1024 rows, 1024 columns, 11264 nonzeros


In [4]:
# Call saucy on the lp graph
saucy_out_file = open("./saucy_gens.txt", "w")
saucy_out = sp.Popen(["./lp_saucy/bin/saucy2-5",symm_lp], stdout = saucy_out_file,
                            stderr = saucy_out_file, shell = False)
saucy_return, saucy_err = saucy_out.communicate()
sage_gens = []
gen_idx = 0
# Read in the orbit generators from saucy
with open("./saucy_gens.txt") as f:
    lines = f.readlines()
    for line in lines:
        if line[0] == "C":
            continue
        else:
            sage_gens.append([])
            orbits = re.findall(r"\((.*?)\)",line)
            for orb in orbits:
                temp = []
                for node in orb.split(" "):
                    if node in col_idx.keys():
                        temp.append(col_idx.get(node) + 1)
                    else:
                        temp.append(row_idx.get(node) + 1)
                temp = tuple(temp)
                sage_gens[gen_idx].append(temp)
            gen_idx += 1

            
# Do sage permutation group from the generators
sage_perm_group = PermutationGroup(sage_gens)
gap_group = lg(sage_perm_group)
orbits_to_write = []
cut_orbits = []




In [5]:
# Get first stabilizer group and orbits
stab_group, real_orbits = get_stabilizer_group_orbits(sage_perm_group)
# Concatenate the orbits from the current stabilizer group
# and from the generated cuts and group action on said cuts
orbits_to_write = concatenate_orbits(real_orbits, cut_orbits)
# Write the orbits to a text file to be read into aggregation code
write_orbits(orbits_to_write)
# Call coin-or cut generator that generates cuts from my aggregate lp
call_coin_cgl(symm_lp)
# Generate symmetrical cuts in the orbit of cuts from coin-or cgl
generate_cut_orbits(gap_group, symm_lp, cut_orbits)
# Run until the orbits are discrete
# while (len(real_orbits) < num_tot):
#     stab_group, real_orbits = get_stabilizer_group_orbits(stab_group)
#     orbits_to_write = concatenate_orbits(real_orbits, cut_orbits)
#     write_orbits(orbits_to_write)
#     call_coin_cgl("./test.mps")
#     generate_cut_orbits(gap_group, "./test.mps", cut_orbits)
all_cuts_added = gp.read("./test.mps")
all_cols = all_cuts_added.getVars()
for col in all_cols:
    col.VType = 'B'
all_cuts_added.optimize() 

New Orbits Calculated
Orbits Concatenated
Orbits Written
Cgl Solved Aggregate LP
Read MPS format model from file ../EQLPSolver/HS-COV-COD/cod103.mps
Reading time = 0.00 seconds
obj: 1024 rows, 1024 columns, 11264 nonzeros
Cut Orbits Generated
Read MPS format model from file ./test.mps
Reading time = 0.19 seconds
obj: 2048 rows, 1024 columns, 1059840 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license